In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
import sys
sys.path.append("/cellarold/users/mpagadal/Programs/anaconda3/lib/python3.7/site-packages")
import re
import myvariant
mv = myvariant.MyVariantInfo()

In [3]:
def get_clumps(directory,snp_type):
    '''
    Input: directory with all plink .clumped files to be aggregated
    Output: dataframe with INDEX SNP, p-value, association paramter
    '''
    #get clumped files
    files=os.listdir(directory)
    file_lst=[x for x in files if ".clumped" in x]
    
    if snp_type == "index":
    
        #iterate through all clumped files and extract index snps
        snps=[]
        assoc=[]
        p=[]

        for x in file_lst:
            file=pd.read_csv(directory+"/"+x, delim_whitespace=True)
            for i,row in file.iterrows():
                snps.append(row["SNP"])
                p.append(row["P"])
                assoc.append(x.split(".clump.clumped")[0])
    
        df=pd.DataFrame({"snps":snps,"file":assoc,"p-value":p})
        return(df)
    
    if snp_type == "all":
        snps=[]
        indexsnp=[]
        assoc=[]
        p=[]

        for x in file_lst:
            file=pd.read_csv(directory+"/"+x, delim_whitespace=True)
            for i,row in file.iterrows():
                for y in row["SP2"].split("(1),"):
                    snps.append(y)
                    indexsnp.append(row["SNP"])
                    p.append(row["P"])
                    assoc.append(x.split(".clump.clumped")[0])
                
        df=pd.DataFrame({"indexsnps":indexsnp,"snps":snps,"file":assoc,"p-value":p})
        df=df[~(df["snps"]=="")]
        df=df[~(df["snps"]=="NONE")]
        df["snps"]=df["snps"].str.split("(").str[0]
        return(df)

In [4]:
def map_rsid(x):
    '''
    Input: SNP format (chr:bp:minor:major)
    Output: tuple (variant,rsid)
    '''
    try:
        if "rs" in x:
            try:
                var=(re.findall('\d+',mv.getvariants(x)[-1]["_id"]))[0]+":"+(re.findall('\d+',mv.getvariants(x)[-1]["_id"]))[1]+":"+re.split("[^a-zA-Z]*",mv.getvariants(x)[-1]["_id"])[-4]+":"+re.split("[^a-zA-Z]*",mv.getvariants(x)[-1]["_id"])[-2]
                rs=x
            except:
                var=""
                rs=x
        else:
            try:
                var="chr"+x.split(":")[0]+":g."+x.split(":")[1]+x.split(":")[2]+">"+x.split(":")[3]
                rs=mv.getvariants(var,fields='dbsnp.rsid')[0]["dbsnp"]["rsid"]
            except:
                try:
                    var="chr"+x.split(":")[0]+":g."+x.split(":")[1]+x.split(":")[3]+">"+x.split(":")[2]
                    rs=mv.getvariants(var,fields='dbsnp.rsid')[0]["dbsnp"]["rsid"]
                except:
                    var="chr"+x.split(":")[0]+":g."+x.split(":")[1]+x.split(":")[3]+">"+x.split(":")[2]
                    rs=""
    except:
        var=""
        rs=""
        
    return (var,rs)
  

In [5]:
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

### Collect variants from previous germline studies

In [6]:
lit_germline=pd.DataFrame()

In [7]:
#FGFR4
kogan=["rs351855"]
kogan_snp=["5:176520243:G:A"]
kogan_df=pd.DataFrame({"snps":kogan,"key":"kogan","association":"FGFR4","variant":kogan_snp})
lit_germline=lit_germline.append(kogan_df)

In [8]:
#CTLA4
queirolo=["rs4553808","rs11571316","rs11571317","rs3087243","rs5742909","rs231775"]
queirolo_snp=["2:204731005:A:G","2:204731089:G:A","2:204732008:C:T","2:204738919:G:A","2:204732347:C:T","2:204732714:A:T"]
queirolo_df=pd.DataFrame({"snps":queirolo,"key":"queirolo","association":"CTLA4","variant":queirolo_snp})
lit_germline=lit_germline.append(queirolo_df)

In [9]:
#IRF5
uccellini=["rs10954213"]
uccellini_snp=["7:128589427:G:A"]
uccellini_df=pd.DataFrame({"snps":uccellini,"key":"uccellini","association":"IRF5","variant":uccellini_snp})
lit_germline=lit_germline.append(uccellini_df)

In [10]:
#CXCR3, CCR5
bedognetti=["rs2280964", "rs333"]
bedognetti_snp=["X:70838054:G:C",np.nan]
bedognetti_df=pd.DataFrame({"snps":bedognetti,"key":"bedognetti","association":"CXCR3/CCR5","variant":bedognetti_snp})
lit_germline=lit_germline.append(bedognetti_df)

In [11]:
#lim et al PNAS
lim=pd.read_csv("../data/yang/pnas.variant.csv")
lim_df=pd.DataFrame({"snps":lim["SNP"].tolist(),"key":"lim","association":lim["Geneset"].tolist(),"variant":lim["variant"].tolist()})
lit_germline=lit_germline.append(lim_df)

In [12]:
#Shahamtdar
shahamtdar=["rs4819959","rs3366"]
shahamtdar_snp=["22:17586631:G:A","21:44834825:G:T"]
shahamtdar_assoc=["Th17","Tfh"]
shahamtdar_df=pd.DataFrame({"snps":shahamtdar,"key":"shahamtdar","association":shahamtdar_assoc,"variant":shahamtdar_snp})
lit_germline=lit_germline.append(shahamtdar_df)

In [13]:
#Zhang
zhang=["rs3903072"]
zhang_snp=["11:65583066:G:T"]
zhang_df=pd.DataFrame({"snps":zhang,"key":"zhang","association":"CTSW","variant":zhang_snp})
lit_germline=lit_germline.append(zhang_df)

In [14]:
#APOE
ostendorf=["rs429358","rs7412"]
ostendorf_snp=["19:45411941:T:C","19:45412079:C:T"]
ostendorf_df=pd.DataFrame({"snps":ostendorf,"key":"ostendorf","association":"APOE","variant":ostendorf_snp})
lit_germline=lit_germline.append(ostendorf_df)

In [15]:
#PD-1
salmaninejad=["rs36084323","rs11568821","rs2227981","rs2227982","rs7421861"]
salmaninejad_snp=["2:242801596:C:T","2:242793912:C:T","2:242793273:A:G","2:242793433:G:A","2:242795350:A:G"]
salmaninejad_df=pd.DataFrame({"snps":salmaninejad,"key":"salmaninejad","association":"PD-1","variant":salmaninejad_snp})
lit_germline=lit_germline.append(salmaninejad_df)

In [16]:
#PD-1
sasaki=["rs36084323"]
sasaki_snp=["2:242801596:C:T"]
sasaki_df=pd.DataFrame({"snps":sasaki,"key":"sasaki","association":"PD-1","variant":sasaki_snp})
lit_germline=lit_germline.append(sasaki_df)

In [17]:
#PD-1
tang=["rs10204525","rs7421861","rs2227982"]
tang_snp=["2:242792321:C:T","2:242795350:A:G","2:242793433:G:A"]
tang_df=pd.DataFrame({"snps":tang,"key":"tang","association":"PD-1","variant":tang_snp})
lit_germline=lit_germline.append(tang_df)

In [18]:
#PD-L1
kula=["rs4143815","rs2297136","rs822336","rs822337","rs2282055","rs2890658","rs10815225","rs2890657","rs822338","rs17718883"]
kula_snp=["9:5468257:G:C","9:5467955:G:A","9:5448690:G:C","9:5449154:T:A","9:5455732:T:A","9:5465130:C:A","9:5450497:G:A","9:5452560:G:C","9:5451557:C:A","9:5462876:C:G"]
kula_df=pd.DataFrame({"snps":kula,"key":"kula","association":"PD-1","variant":kula_snp})
lit_germline=lit_germline.append(kula_df)

In [19]:
#PD-L1
yoshida=["rs822339","rs1411262"]
yoshida_snp=["9:5453172:A:G","9:5459419:C:T"]
yoshida_df=pd.DataFrame({"snps":yoshida,"key":"yoshida","association":"PD-L1","variant":yoshida_snp})
lit_germline=lit_germline.append(yoshida_df)

In [20]:
#Sayaman Immunity
sayaman=pd.read_excel("../data/sayaman/1-s2.0-S1074761321000340-mmc5.xlsx",sheet_name=1)
sayaman["snp_noallele"]=sayaman["PLINK.SNP"].str.rsplit(":",2).str[0]
sayaman_sig=sayaman[sayaman["Annot.Figure.GenomewideSignificant.SNP"]=="GW Significant"]
sayaman_snps=sayaman_sig["PLINK.SNP"].unique().tolist()
print(len(sayaman_snps))

584


#### conduct clumping for sayaman

In [21]:
bim=pd.read_csv("/cellar/controlled/dbgap-genetic/phs000178_TCGA/imputation/michigan-imputation/HRC/european.final.noimmunecancers.clean.bim",header=None,delim_whitespace=True)
bim["snp_noallele"]=bim[1].str.rsplit(":",2).str[0]

In [22]:
for x in sayaman_sig["Sayaman.InternalLabel"].unique():
    print(x)
    assoc=sayaman_sig[sayaman_sig["Sayaman.InternalLabel"]==x]
    snps=assoc["PLINK.SNP"].tolist()+[x.rsplit(":",2)[0]+":"+x.split(":")[3]+":"+x.split(":")[2] for x in assoc["PLINK.SNP"].tolist()]
    assoc_bim=bim[bim[1].isin(snps)]
    assoc=pd.merge(assoc_bim, assoc[["snp_noallele","PLINK.P","Sayaman.InternalLabel"]],on="snp_noallele",how="left")
    print(assoc.shape)
    if len(assoc)>0:
        assoc=assoc[[1,"PLINK.P"]]
        assoc.columns=["SNP","P"]
        assoc.to_csv("../data/sayaman/supp-associations/"+x+".assoc",index=None,sep="\t")

Sigs160.Bindea_T.helper.cells
(0, 9)
Sigs160.Wolf_IFN.21978456
(6, 9)
Sigs160.Wolf_Interferon.19272155
(3, 9)
Sigs160.Attractors_IFIT3
(2, 9)
Sigs160.Bindea_CD8.T.cells
(0, 9)
Sigs160.Wolf_Module3.IFN.score
(5, 9)
Sigs160.Wolf_Interferon.Cluster.21214954
(1, 9)
Sigs160.Senbabaoglu_APM1
(0, 9)
Sigs160.Bindea_Tfh.cells
(2, 9)
Sigs160.Wolf_CD8.CD68.ratio
(6, 9)
Sigs160.Wolf_MHC2.21978456
(522, 9)
Sigs160.Bindea_aDC
(1, 9)
Sigs160.Bindea_NK.cells
(0, 9)
Sigs160.Bindea_Cytotoxic.cells
(2, 9)
Sigs160.Wolf_LYMPHS.PCA.16704732
(0, 9)
Core56.Cell.Proportion_Eosinophils_Binary.MedianLowHigh
(7, 9)
Sigs160.Bindea_Eosinophils
(0, 9)
Sigs160.Bindea_Tcm.cells
(0, 9)
Sigs160.Bindea_Th17.cells
(19, 9)


run script: **sayaman-clumping.sh**

In [23]:
sayaman_clump=get_clumps("../data/sayaman/supp-associations/","index")

In [24]:
sayaman["variant"]=sayaman["PLINK.SNP"]
sayaman["variant2"]=sayaman["PLINK.SNP"].str.rsplit(":",2).str[0]+":"+sayaman["PLINK.SNP"].str.split(":").str[3]+":"+sayaman["PLINK.SNP"].str.split(":").str[2]

mp_beta=Merge(dict(zip(sayaman["variant"],sayaman["PLINK.BETA.OR"])),dict(zip(sayaman["variant2"],sayaman["PLINK.BETA.OR"])))
mp_a1=Merge(dict(zip(sayaman["variant"],sayaman["PLINK.A1"])),dict(zip(sayaman["variant2"],sayaman["PLINK.A1"])))

In [25]:
sayaman_clump["beta"]=sayaman_clump["snps"].map(mp_beta)
sayaman_clump["A1"]=sayaman_clump["snps"].map(mp_a1)

In [26]:
sayaman_clump.shape

(55, 5)

In [27]:
sayaman_clump=sayaman_clump.rename(columns={"snps":"variant","file":"association"})

In [28]:
sayaman_clump["key"]="sayaman"

In [29]:
lit_germline=lit_germline.append(sayaman_clump)

/cellar/users/mpagadal/Programs/miniconda3/envs/baseold/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


### make literature review snps dataframe

##### map the rsid to coord

In [30]:
lit_germline['mv_variant'] = lit_germline['snps'].apply(lambda x: map_rsid(x)[0])
lit_germline['mv_rsid'] = lit_germline['variant'].apply(lambda x: map_rsid(x)[1])

querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1...done.
querying 1-1.

In [32]:
lit_germline["snps"]=lit_germline["snps"].fillna(lit_germline["mv_rsid"])
lit_germline["variant"]=lit_germline["variant"].fillna(lit_germline["mv_variant"])

In [33]:
lit_germline=lit_germline[["variant","snps","association","key","A1","beta","p-value"]]

In [38]:
lit_germline.columns=["variant","rsid","file","lit_source","A1","beta","p-value"]

In [41]:
lit_germline.to_csv("../data/supplemental/Supplemental_Table_5.csv",index=None,sep="\t")

In [40]:
lit_germline["lit_source"].value_counts()

lim             103
sayaman          55
kula             10
queirolo          6
salmaninejad      5
tang              3
shahamtdar        2
bedognetti        2
yoshida           2
ostendorf         2
zhang             1
sasaki            1
uccellini         1
kogan             1
Name: lit_source, dtype: int64